In [1]:
!pip install python_speech_features
!pip install scipy

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=80c7b51b1c0f1324916e28270b047c7cb9a76d6d6522099003468930fe8d56f3
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\09\a1\04\08e2688d2562d8f9ff89e77c6ddfbf7268e07dae1a6f22455e
Successfully built python-speech-features


# Importing relevant libraries

In [31]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator

In [32]:
# define a function to get distance between feature vectors and find neighbours
def getNeighbours(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = distance(trainingset[x], instance,k) + distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2],dist))
    distances.sort(key=operator.itemgetter(1))
    neighbours = []
    for x in range(k):
        neighbours.append(distances[x][0])
    return neighbours

In [33]:
# Function to identify nearest neighbours
def nearClass(neighbours):
    classVote = {}
    for x in range(len(neighbours)):
        response = neighbours[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
            
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [34]:
def getAccuracy(testset,prediction):
    correct = 0
    for x in range(len(testset)):
        if testset[x][-1] == prediction[x]:
            correct += 1
    return 1.0 * correct / len(testset)

In [35]:
directory = 'C:\\Users\\DELL\\Downloads\\archive\\Data\\genres_original'
f = open('mydataset.dat',"wb")
i  = 0
for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+"/"+folder):
        try:
            (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
            mfcc_feat = mfcc(sig,rate,winlen = 0.020, appendEnergy = False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix,covariance,i)
            pickle.dump(feature,f)
        except Exception as e:
            print("Got an exception:",e,'in folder:',folder,'filename:',file)
f.close()

In [36]:
dataset = []
def loadDataset(filename,split,trset,teset):
    with open('mydataset.dat','rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trset.append(dataset[x])
        else:
            teset.append(dataset[x])
            
trainingSet = []
testSet = []
loadDataset('mydataset.dat',0.68,trainingSet,testSet)
loadDataset

<function __main__.loadDataset(filename, split, trset, teset)>

In [43]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2),cm1))
    distance += (np.dot(np.dot((mm2 - mm1).transpose(),np.linalg.inv(cm2)),mm2 - mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [46]:
# Make the prediction using KNN(K-nearest)
length = len(testSet)
predictions = []
for x in range(length):
    predictions.append(nearClass(getNeighbours(trainingSet, testSet[x], 5)))
    
accuracy1 = getAccuracy(testSet,predictions)
accuracy1

0.6838905775075987

In [41]:
from collections import defaultdict
results = defaultdict(int)
directory = 'C:\\Users\\DELL\\Downloads\\archive\\Data\\genres_original'

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [45]:
pred = nearClass(getNeighbours(dataset,feature,5))
results[pred]

'rock'